## OpenAI Agent SDK Demo

Core concepts:
- Agents: LLMs configured with instructions, tools, guardrails, and handoffs
- Handoffs: A specialized tool call used by the Agents SDK for transferring control between agents
- Guardrails: Configurable safety checks for input and output validation
- Sessions: Automatic conversation history management across agent runs
- Tracing: Built-in tracking of agent runs, allowing you to view, debug and optimize your workflows

The agent loop:  
When you call Runner.run(), we run a loop until we get a final output.

We call the LLM, using the model and settings on the agent, and the message history.
The LLM returns a response, which may include tool calls.
If the response has a final output (see below for more on this), we return it and end the loop.
If the response has a handoff, we set the agent to the new agent and go back to step 1.
We process the tool calls (if any) and append the tool responses messages. Then we go to step 1.
There is a max_turns parameter that you can use to limit the number of times the loop executes.

In [1]:
!pip install openai-agents


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: this example isn't exactly an agent as it has no tool calls but this is just for an example.

In [6]:
from agents import Agent, Runner
from dotenv import load_dotenv
from dataclasses import dataclass

load_dotenv()

@dataclass
class UserContext:
    name: str
    uid: str
    is_pro_user: bool

# define the agents
agent = Agent[UserContext](name="Assistant", 
              instructions="You are a helpful assistant")

# get the runner to loop
result = await Runner.run(agent, 
                          "What is a dog?")
print(result.final_output)

A **dog** is a domesticated mammal and a subspecies of *Canis lupus* (the gray wolf) with the scientific name **Canis lupus familiaris**. Dogs have been bred by humans for thousands of years for a variety of purposes, including companionship, work (such as herding, guarding, and hunting), and help in various roles (such as guide dogs and police dogs).

**Key features of dogs:**
- Four-legged, furry animals with varied sizes, shapes, and coat types.
- Highly social and often form strong bonds with humans.
- Intelligent, trainable, and able to understand commands and cues.
- Known for keen senses — especially smell and hearing.
- Typically communicate through barking, whining, body language, and tail movements.

Dogs are one of the most popular pets worldwide and are valued for their loyalty, playfulness, and ability to adapt to human environments.


Handoffs example

In [ ]:
from agents import Agent, Runner

spanish_agent = Agent(
    name="Spanish agent",
    instructions="You only speak Spanish.",
)

english_agent = Agent(
    name="English agent",
    instructions="You only speak English",
)

# could say this is the principle agent that handles the other 
# task agents as tool calls almost (agent handoff)
triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[spanish_agent, english_agent],
)

result = await Runner.run(triage_agent, 
                          input="Hola, ¿cómo estás?",
                          max_turns=5   # limit conversation loop to 5 turns
                        )
# CAN ADD MAX TURNS!!!!!!!!!!! THIS LIMITS THE AMOUNT OF LOOPS WE DO

print(result.final_output)

¡Hola! Estoy muy bien, gracias. ¿Y tú? ¿En qué puedo ayudarte hoy?


Functions example

In [9]:
from agents import Agent, Runner, function_tool

@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny."

agent = Agent(
    name="Hello world",
    instructions="You are a helpful agent.",
    tools=[get_weather],
)

result = await Runner.run(agent, input="What's the weather in Tokyo?")
print(result.final_output)
# The weather in Tokyo is sunny.

The weather in Tokyo is sunny. Would you like more details, such as the temperature or forecast for the week?


Sessions
--------
The Agents SDK provides built-in session memory to automatically maintain conversation history across multiple agent runs, eliminating the need to manually handle .to_input_list() between turns.

In [5]:
from agents import Agent, Runner, SQLiteSession

# Create agent
agent = Agent(
    name="Assistant",
    instructions="Reply very concisely.",
)

# Create a session instance
session = SQLiteSession("conversation_123")

# First turn
result = await Runner.run(
    agent,
    "What city is the Golden Gate Bridge in?",
    session=session
)
print(result.final_output)  # "San Francisco"

# Second turn - agent automatically remembers previous context
result = await Runner.run(
    agent,
    "What state is it in?",
    session=session
)
print(result.final_output)  # "California"

San Francisco.
California.


Session options
No memory (default): No session memory when session parameter is omitted
session: Session = DatabaseSession(...): Use a Session instance to manage conversation history

In [6]:
from agents import Agent, Runner, SQLiteSession

agent = Agent(name="Assistant")

# Different session IDs maintain separate conversation histories
result1 = await Runner.run(
    agent,
    "What's the population?",
    session=session
)

# Custom SQLite database file
session1 = SQLiteSession("user_123", "conversations.db")
result2 = await Runner.run(
    agent,
    "What's the population?",
    session=session1
)

result3 = await Runner.run(
    agent,
    "What is a dog?",
    session=SQLiteSession("user_456", "conversations.db")
)

result4 = await Runner.run(
    agent,
    "Tell me more",
    session=session1
)

print(result1.final_output)
print(result2.final_output)
print(result3.final_output)
print(result4.final_output)

As of the 2020 United States Census, the population of **San Francisco, California** was approximately **815,201** people. More recent estimates (as of 2023) put the population at around **808,000**.
Could you please clarify what location or entity you’re referring to? For example, are you asking about the population of a specific country, city, or region? Let me know so I can give you the most accurate and up-to-date information!
A **dog** is a domesticated mammal with the scientific name **Canis lupus familiaris**. Dogs are closely related to wolves and are believed to have been domesticated by humans more than 15,000 years ago. They are known for their loyalty, companionship, intelligence, and a wide range of breeds with different physical and behavioral traits.

**Key facts about dogs:**
- **Classification:** Mammal, in the family Canidae.
- **Diet:** Omnivorous, but primarily carnivorous.
- **Lifespan:** Varies by breed, typically 10-15 years.
- **Roles:** Companion animal, workin

Custom session implementations
You can implement your own session memory by creating a class that follows the Session protocol (i.e. maybe you have a certain way you do your history truncating):

EXAMPLE IN DOCUMENTATION